In [ ]:
import pyaudio
import numpy as np
import wave

# Parameters
CHUNK = 1024  # Number of audio frames per buffer
FORMAT = pyaudio.paInt16  # Audio format (16-bit PCM)
CHANNELS = 1  # Mono audio
RATE = 44100  # Sampling rate (44.1 kHz)
VOLUME_REDUCTION_FACTOR = 0.5  # Reduce volume by 50%
OUTPUT_FILENAME = "./Output/mic_captured_audio.wav"  # File to save audio

# Initialize PyAudio
p = pyaudio.PyAudio()

# List all audio devices
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

# Open input stream (microphone)
input_stream = p.open(format=FORMAT,
                      channels=CHANNELS,
                      rate=RATE,
                      input=True,
                      input_device_index=1,  # Replace input device index with your mic device index
                      frames_per_buffer=CHUNK)

# Open output stream (speakers)
output_stream = p.open(format=FORMAT,
                       channels=CHANNELS,
                       rate=RATE,
                       output=True,
                       frames_per_buffer=CHUNK)

# Prepare to save audio to file
wave_file = wave.open(OUTPUT_FILENAME, 'wb')
wave_file.setnchannels(CHANNELS)
wave_file.setsampwidth(p.get_sample_size(FORMAT))
wave_file.setframerate(RATE)

print("Capturing audio... Press Ctrl+C to stop.")

try:
    while True:
        # Read audio data from microphone
        data = input_stream.read(CHUNK)
        
        # Convert audio data to NumPy array
        audio_data = np.frombuffer(data, dtype=np.int16)
        
        # Reduce volume
        audio_data = (audio_data * VOLUME_REDUCTION_FACTOR).astype(np.int16)
        
        # Convert NumPy array back to bytes
        output_data = audio_data.tobytes()
        
        # Play audio through speakers
        output_stream.write(output_data)
        
        # Save original audio data to file
        wave_file.writeframes(data)
except KeyboardInterrupt:
    print("Audio stream stopped.")
finally:
    # Ensure streams are properly closed
    if input_stream.is_active():
        input_stream.stop_stream()
    input_stream.close()
    
    if output_stream.is_active():
        output_stream.stop_stream()
    output_stream.close()
    
    wave_file.close()
    p.terminate()
    print("Resources released successfully.")